<div style="text-align: right"> 

### DATA 22100 - Introduction to Machine Learning

</div>

<img src="https://raw.githubusercontent.com/david-biron/DATA221imgs/main/UChicago_DSI.png" align="right" alt="UC-DSI" width="300">







<center> 

# (Simple) Recurrent Neural Networks
    
<br/>
    
</center> 

    

### RNNs vs. Feed-forward networks

|   |   |  |
|:-|:-:|:-|
| **Feed-forward netwok** | &nbsp; &nbsp; &nbsp; | **Recurrent Network** | 
| Fixed input size (e.g., image)  | | Variable input size |
| All input taken in at once      | | Input sequence taken one vector at a time |
| Fixed output size (e.g., class) | | Variable output size |  

<br/> 

### At each step the RNN considers previous steps through the hidden state 

* A **hidden state** vector is initialized to zeros. 
* At each step, the RNN considers an input vector (one of a sequence of input vectors) along with the previous hidden state.
* The previous hidden state and the input vector are both used by the RNN cell to calculate the next hidden state (and output). 
* The final result **depends on all the previous input vectors** through the hidden states that were passed along.
* A typical caculation, e.g., performed by [`torch.nn.RNN`](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) is:

$$\begin{eqnarray} 
\text{next\_hidden} &=& \tanh\left[\text{input\_item} \times \text{input\_weight(s)} + \text{input\_bias(es)} \right. \\
&& \hspace{10mm} \left. + \text{previous\_hidden} \times \text{hidden\_weight(s)} + \text{hidden\_bias(es)}  \right]
\end{eqnarray}$$ 

Or, adopting the terminology of **time-steps**:

$$\begin{eqnarray} \text{hidden}_t &=&  
\tanh\left[\text{input}_{t} \  \text{weights}_{\text{input}} + \text{biases}_{\text{input}} + \text{hidden}_{t-1} \ \text{weights}_{\text{hidden}} + \text{biases}_{\text{hidden}}  \right] 
\end{eqnarray}$$

* The argument of the $\tanh(.)$ function is a linear layer. 
* There are variations on this theme. For instance, $ReLu(.)$ can replace the $\tanh(.)$ 

<br> 

<img src="https://raw.githubusercontent.com/david-biron/DATA221imgs/main/RNN_animation.gif" width="300">


<br> 

#### 'Unrolled' diagrams of RNNs are common

<img src="https://raw.githubusercontent.com/david-biron/DATA221imgs/main/RNN_diagram.png" width="450">

This can be somewhat confusing. **Note:** 

The **same** RNN cell is reused throughout: <br> only one set of weights (and biases) is updated.

<br>




| |  |
|:-:|:-| 
|<img src="https://raw.githubusercontent.com/david-biron/DATA221imgs/main/icon_example.png" width="50">| A contrived example with arbitrary numbers. Each niput item is a vector with two elements. <br> A sigle output is calculated after the a sequence of two inputs. Red: input. Yellow: current hidden state. <br> Green: next hidden state. For simplicity: hidden state weights are $1$ and biases are $0$ (not shown). | 

<img src="https://raw.githubusercontent.com/david-biron/DATA221imgs/main/RNNcontrivedExample.png" width="500">


#### Different tasks may require sequences of inputs, output, or both

<img src="https://raw.githubusercontent.com/david-biron/DATA221imgs/main/RNN_types.png" width="600">

* Each rectangle is a vector 
* Input vectors are red
* Output vectors are blue
* Hidden state vectors are green

#### Recomended reading 
[The Unreasonable Effectiveness of Recurrent Neural Networks (2015)](https://karpathy.github.io/2015/05/21/rnn-effectiveness/)




### The goal:
"You give it a large chunk of text and it will learn to generate text like it **one character at a time**."

We'll frame this like a **classification task**: 
* The desired product of the model (after considering an entire sequence of characters) is a single output - the most likely next character. 
* Each possible character (e.g., lowercase letter or space) will be a class. 
* The model will produce a vector of class probability scores from which we can choose the most likely class/character. 


<br/> 

#### Supervised learning

During training:
* Each sequence of training characters (represented by a sequence of vectors) has a corresponding target character or sequence of characters that the model should learn. 
* Forward pass $\rightarrow$ prediction and loss
* Backward pass $\rightarrow$ gradient 
* Optimizer step $\rightarrow$ update weights 
* Zero the gradient, etc. 

<br/> 

#### One-hot-encoding: a naive vector represention of characters 

|   |   | 
|:-:|:-:|
| <img src="https://raw.githubusercontent.com/david-biron/DATA221imgs/main/icon_example.png" width="50"> | One-hot-encoding for a list of four characters: \[ 'w', 'x', 'y', 'z' \] | 
| 'w' | \[ 1, 0 , 0, 0 \] |
| 'x' | \[ 0, 1, 0 , 0 \] |
| 'y' | \[ 0, 0, 1 , 0 \] |
| 'z' | \[ 0, 0, 0 , 1 \] |





In [1]:
from sklearn.preprocessing import OneHotEncoder

char_list = [[c] for c in 'wxyz'] # will get converted to 2d-array
enc = OneHotEncoder() 
print(enc.fit_transform(char_list).toarray())


[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


|   |   | 
|:-:|:-|
| <img src="https://raw.githubusercontent.com/david-biron/DATA221imgs/main/icon_comment.png" width="50"> | A more sophisticated approach could treat each **word** as a single  <br> token and use word embedding, e.g., [BERT word embedding](https://en.wikipedia.org/wiki/BERT_(language_model)), <br> to encode them as vectors. We will not implement that here <br> (the examples below use smaller data and a simpler model). |

### Implementing a simple RNN manually (sort of)

`PyTorch` implements a [multilayer RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) but for startyers we'll implement our own. 



In [22]:
# !pip3 install torchvision # ==0.14.1
# !pip3 install torch # ==1.13.1 



In [2]:
import numpy as np
import pandas as pd
from torch import nn
import torch
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
from sklearn.preprocessing import OneHotEncoder




### Individual input items 

Individual lowercase alphabetic characters, a space character, and "'". 

All 28 characters are one-hot encoded (each represented by a vector).  

In [3]:
chars = " 'abcdefghijklmnopqrstuvwxyz"
char_list = [[c] for c in chars] # will get converted to 2d-array
enc = OneHotEncoder() 
enc.fit(char_list)

# Being able to go back from a vector 
# to a characterwill be useful: 
def vec_to_char(vec, enc): 
    '''
    Input: one-hot encoded vector and the encoder. 
    Output: corresponding character from 
            " 'abcdefghijklmnopqrstuvwxyz"
    '''
    if isinstance(vec, np.ndarray):
        idx = np.argmax(vec)
    elif isinstance(vec, torch.Tensor): 
        idx = torch.argmax(vec).item()
    else: 
        return None 
    return enc.categories_[0][idx]

# Test the encoding and decoding: 
one_hot = enc.transform(char_list).toarray()
print(one_hot[2],'-->',vec_to_char(one_hot[2], enc))


[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.] --> a


### The training data: a para from a (great) book
#### (somewhat cleaned)




In [4]:
text = "It was like the second when you come home late at night and "
text += "see the yellow envelope of the telegram sticking out from "
text += "under your door and you lean and pick it up, "
text += "but don't open it yet, not for a second. While you stand "
text += "there in the hall, with the envelope in your hand, "
text += "you feel there's an eye on you, a great big eye looking "
text += "straight at you from miles and dark and through walls "
text += "and houses and through your coat and vest and hide and "
text += "sees you huddled up way inside, in the dark which is you, "
text += "inside yourself, like a clammy, sad little foetus you "
text += "carry around inside yourself. The eye knows "
text += "what's in the envelope, and it is watching you to see you "
text += "when you open it and know, too. But the clammy, sad little "
text += "foetus which is you way down in the dark which is you too "
text += "lifts up its sad little face and its eyes are blind, "
text += "and it shivers cold inside you for it doesn't want to "
text += "know what is in that envelope. It wants to lie in the "
text += "dark and not know, and be warm in its not-knowing. "

text += "The end of man is knowledge, but there is one thing "
text += "he can't know. He can't know whether knowledge will "
text += "save him or kill him. He will be killed, all right, "
text += "but he can't know whether he is killed because of the "
text += "knowledge which he has got or because of the knowledge "
text += "which he hasn't got and which if he had it, would save "
text += "him. There's the cold in your stomach, but you open the "
text += "envelope, you have to open the envelope, for the end of "
text += "man is to know."

text = text.lower()
text = text.replace('.','')
text = text.replace(',','')
text = text.replace('-','')
# set(''.join(text))


### Simple RNN using `torch.nn.RNN`

**For simplicity:**  

* Divide the training text to fixed length sequences of characters. 
* Pad the last sequence with spaces as needed. 

In [5]:
chars = " 'abcdefghijklmnopqrstuvwxyz"
int2char = dict(enumerate(chars))
char2int = {char: ind for ind, char in int2char.items()}



In [6]:
N = 20
tmp = [(text[c:c+N]) 
        for c in range(0, len(text), N)]
tmp[-1] = tmp[-1]+' '*(N-len(tmp[-1]))

X_train = [x[:-1] for x in tmp]
Y_train = [y[1:] for y in tmp]

print(X_train[:3])
print(Y_train[:3])

['it was like the sec', 'nd when you come ho', 'e late at night and']
['t was like the seco', 'd when you come hom', ' late at night and ']


In [7]:
for i in range(len(X_train)):
    X_train[i] = [char2int[c] for c in X_train[i]]
    Y_train[i] = [char2int[c] for c in Y_train[i]]

print(X_train[0])
print(Y_train[0])

[10, 21, 0, 24, 2, 20, 0, 13, 10, 12, 6, 0, 21, 9, 6, 0, 20, 6, 4]
[21, 0, 24, 2, 20, 0, 13, 10, 12, 6, 0, 21, 9, 6, 0, 20, 6, 4, 16]


#### One-hot encode each input sequence


In [29]:
N_chars = len(chars)
seq_len = N - 1
batch_size = len(X_train)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    '''
    Input: a list of sequences of integers (of a fixed length).
    Output: a torch.tensor with the one-hot encoded sequences. 
    '''
    vec = np.zeros((batch_size, seq_len, dict_size), 
                    dtype=np.float32)
    for i in range(batch_size):
        for u in range(seq_len):
            vec[i, u, sequence[i][u]] = 1
    return torch.tensor(vec)

X_train_enc = one_hot_encode(X_train, N_chars, seq_len, batch_size)

Y_train_enc = torch.Tensor(Y_train)

print(X_train_enc[0][0])  # 'i'
print(X_train_enc[0][8])  # 'i' 
print(Y_train_enc[1][4])  # 'e' 
print(Y_train_enc[1][14]) # 'e'
print(type(X_train_enc), type(Y_train_enc))
print(X_train_enc.shape, Y_train_enc.shape)


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor(6.)
tensor(6.)
<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([70, 19, 28]) torch.Size([70, 19])


#### A model with a single `torch.nn.RNN` layer:

In [30]:
class myRNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(myRNN, self).__init__()

        # Create hidden_size attribute
        self.hidden_size = hidden_size

        # Create one RNN Layer
        self.rnn = nn.RNN(input_size=input_size, 
                          hidden_size=hidden_size, 
                          num_layers=1, 
                          batch_first=True)   
        # Linear layer
        self.linear = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input (see below)
        # hidden = self.init_hidden(batch_size)
        hidden = torch.zeros(1, batch_size, self.hidden_size)
        
        # input + hidden state --> outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs for the linear layer
        # (contiguous in memory for performance:
        #  improving data locality improves performance because 
        #  memory access is more efficient)
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.linear(out)
        
        return out, hidden



#### Instantiate the model, ser hyperparameters, choose loss function and optimizer

In [36]:
# Instantiate the model with dimensions:
# Each input: single one-hot-encoded character (length len(chars))
# Each output: single one-hot-encoded character (length len(chars))
# Each hidden state: a tensor of length 12 (arbitrary)
# n_layers=1: a single RNN layer (in principle they could be stacked)

model = myRNN(input_size=len(chars), 
              output_size=len(chars), 
              hidden_size=12)

# Define hyperparameters
n_epochs = 2000
lr=0.01

# Loss function: CrossEntropyLoss() is populaar for classification
loss_func = nn.CrossEntropyLoss() 

# Optimizer: Adam, AdamW, or SGD are popular choices 
optimizer = torch.optim.Adam(model.parameters(), lr=lr)



#### Train the RNN 

In [48]:
out, hid = model(X_train_enc)
print('Output shape:',out.shape)
print('Y_train_enc shape:', Y_train_enc.shape)
print('Y_train_enc.view(-1) shape:', Y_train_enc.view(-1).shape)


Output shape: torch.Size([1330, 28])
Y_train_enc shape: torch.Size([70, 19])
Y_train_enc.view(-1) shape: torch.Size([1330])


In [37]:
%%time 

for epoch in range(n_epochs):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    output, hidden = model(X_train_enc)
    loss = loss_func(output, Y_train_enc.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%100 == 0:
        print('Epoch: {}/{}... Loss: {:.4f}'.format(epoch, 
                                                    n_epochs,
                                                    loss.item()))        
        

Epoch: 0/2000... Loss: 3.4294
Epoch: 100/2000... Loss: 1.8656
Epoch: 200/2000... Loss: 1.4681
Epoch: 300/2000... Loss: 1.2910
Epoch: 400/2000... Loss: 1.2269
Epoch: 500/2000... Loss: 1.1820
Epoch: 600/2000... Loss: 1.1653
Epoch: 700/2000... Loss: 1.1617
Epoch: 800/2000... Loss: 1.1259
Epoch: 900/2000... Loss: 1.1139
Epoch: 1000/2000... Loss: 1.1114
Epoch: 1100/2000... Loss: 1.0937
Epoch: 1200/2000... Loss: 1.1493
Epoch: 1300/2000... Loss: 1.0824
Epoch: 1400/2000... Loss: 1.0773
Epoch: 1500/2000... Loss: 1.0530
Epoch: 1600/2000... Loss: 1.0563
Epoch: 1700/2000... Loss: 1.0479
Epoch: 1800/2000... Loss: 1.0347
Epoch: 1900/2000... Loss: 1.0448


In [49]:
# From model and character predict the next character and hidden state
def predict(model, char_seq):
    # One-hot encoding our input to fit into the model
    char_seq = np.array([[char2int[c] for c in char_seq]])
    char_seq = one_hot_encode(char_seq, N_chars, 
                              char_seq.shape[1], 1)
    
    out, hidden = model(char_seq)

    # Choose class with the highest probability from the output
    prob = nn.functional.softmax(out[-1], dim=0).data
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

# Return sentence from desired output length and input characters
def sample(model, out_len, start):

    # model.eval() is a 'switch' for layers/parts of the model that 
    #              behave differently during training and predicting
    #              (e.g., Dropouts Layers). It is common practice to
    #              predict with torch.no_grad() and model.eval() 
    model.eval() # eval mode

    with torch.no_grad():
        start = start.lower()
        # Run through the starting characters
        chars = [ch for ch in start]
        size = out_len - len(chars)
        
        # Pass in the previous characters and get a new one
        for _ in range(size):
            new_char, _ = predict(model, chars)
            chars.append(new_char)

    return ''.join(chars)

sample(model, 25, "do you")



'do you ope you ope in the'

Not quite English but the model is extremely simple and the training dataset is ridiculously small... 

### The vanishing/exploding gradient problem 
#### (or why simple RNNs are not very useful in and of themselves) 

Generally, a 'vanishing/exploding gradient problem' occurs when the gradients become too small or too large during backpropagation. 

* Gradient too small $\rightarrow$ gradient descent cannot effectively proceed (not to mention underflow). 
* Gradient too large $\rightarrow$ gradient descent cannot effectively converge (not to mention overflow). 

#### RNNs are prone to this problem because of the recurrent connections. 

* Consider a somewhat long (but not unheard of) sequence of $100$ input items. 
* Consider a wieght, $w$, that gets reused $100$ times as the RNN is 'unrolled'.  
* $\frac{d \ loss}{d \ weight} \to 0 \ \ \ \text{ or } \infty$


In [39]:
sample(model, 20, "do you")

'do you ope you ope i'

In [40]:
sample(model, 100, "do you")

'do you ope you ope in the yo  ouro enkye lope in the hall the yes wide in the envelope in your and i'